In [1]:
fileName = 'Equations\Clean\eq1_hr.jpg'
from skimage.color import rgb2gray
import cv2
from scipy.ndimage import interpolation as inter

import numpy as np
import os
os.getcwd()

'C:\\Users\\KINSHUK\\Documents\\GitHub\\LaTeX-Generation-from-Printed-Equation\\actual_code'

In [2]:
def cart2pol(x, y):
    rho = np.sqrt(x**2 + y**2)
    phi = np.arctan2(y, x)
    return(phi, rho)

In [3]:
def create_identifier(img):
    img = img/255
    k = 8;
#     IN2, R1-R7, D2-D8, 6 Hu Moments
    identifier = np.zeros((1,2*k+6));
    img_shape = img.shape
    img_inv = np.ones(img_shape) - img
    pad_size = 3
    
#     Top
    if(sum(img_inv[0,:])==0):
        add = np.ones((pad_size,img_shape[1]))
        img = np.concatenate((add,img))
    elif(sum(img_inv[img_shape[0]-1,:])==0) :#Bottom
        add = np.ones((pad_size,img_shape[1]))
        img = np.concatenate((img, add))
        
    elif(sum(img_inv[:,1])==0) :#left
        add = np.ones((img_shape[0], pad_size))
        img = np.hstack(( add, img))
    
    elif(sum(img_inv[:,1])==0) :#right
        add = np.ones((img_shape[0], pad_size))
        img = np.hstack(( img, add))
    
    #Recalculte Centroid
    img_inv = np.ones(img_shape) - img
    
    N = img.shape[0]*img.shape[1]
    x,y = np.where(img == 0)
    
    
    #Find centroid 
    #gib code kinshuk
    M = cv2.moments(img_inv)
    cent_x = int(M["m10"] / M["m00"])
    cent_y = int(M["m01"] / M["m00"])
    print(cent_x, cent_y)
    blah = np.copy(img_inv)
    cv2.circle(blah, (cent_x, cent_y), 7, (255, 0, 0), -1)
    
    display_image(blah,'Inverse image centroids')
    
#     Find central moment of inertia
    I = sum(np.squeeze((x - cent_x)**2 + (y - cent_y)**2));
    IN2 = I / N**2;
    identifier[0,0] = IN2
    
    x = img_shape[0]
    y = img_shape[1]
    
    dr = max(max(x - cent_x, cent_x), max(y - cent_y, cent_y))/(k+1)
    c,r = np.meshgrid(np.arange(x),np.arange(y))
    c = c + 1
    r = r + 1
    circvec = []
    
    coding = {}
    
    for i in range(1,k+1):
        rad = dr*i
        
        C = (((c - cent_x)**2 + (r - cent_y)**2)**0.5<=rad)^(((c - cent_x)**2 + (r - cent_y)**2)**0.5<=(rad-1))
        
#         print(np.count_nonzero((C+0) == 1))
        indx,indy = np.nonzero(C)
        indx = indx + 1
        indy = indy + 1
        cidx = (indx-1)*C.shape[0] + indy
#         print("cidx", cidx)
        vals = []
        cx = c[indx-1,indy-1] - cent_x
        cy = r[indx-1,indy-1] - cent_y
        for m in range(len(cidx)):
            vals.append([cidx[m], cy[m], cx[m],0 ])
        vals = np.array(vals)
        TH,_ = (cart2pol(vals[:,1], vals[:,2]))
#         print(TH)
        vals[:,3] = TH

#         print(vals)

        sortedvals =  vals[np.argsort(vals[:, 3])]
#         print(sortedvals)
#         C = (C + 0)*255
#         C = C.astype(np.uint8)
#         cv2.imshow("sfsfsf",C)
#         display_image((C+0)*255,"hvhv")
        print(np.unravel_index(sortedvals[:,0], img.shape, 'F'))
        circvec = img[np.unravel_index(sortedvals[:,0], img.shape, 'F')]
        kernel = np.ones((2,1))
        
        print(circvec)
        
        circvec = cv2.morphologyEx(circvec, cv2.MORPH_OPEN, kernel)
        
        coding[i] = circvec
        
        if(len(circvec)!=0):
            circvec = np.squeeze(np.array(circvec))
            dummy = np.array([1,1])
            circvec = np.hstack((dummy, circvec))
            count = repr(circvec).count("0., 0.")
            
            identifier[0,i] = count
    
    li = cv2.HuMoments(cv2.moments(img_inv)).flatten()
    print(li)       
    print(identifier)

In [4]:
def display_image(image,title):
    scaleX = 0.5
    scaleY = 0.5
    scaleDown = cv2.resize(np.copy(image), None, fx= scaleX, fy= scaleY, interpolation= cv2.INTER_LINEAR)
    cv2.imshow(title, scaleDown)
    cv2.waitKey()
    cv2.destroyAllWindows()

In [5]:
class Segments:
    def __init__(self, image_val, boundingbox,centroids):
        self.image_val=image_val
        self.boundingbox = boundingbox
        self.centroids = centroids

In [6]:
def correct_skew(image, delta=1, limit=10):
    def determine_score(arr, angle):
        data = inter.rotate(arr, angle, reshape=False, order=0)
        histogram = np.sum(data, axis=1)
        score = np.sum((histogram[1:] - histogram[:-1]) ** 2)
        return histogram, score

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1] 

    scores = []
    angles = np.arange(-limit, limit + delta, delta)
    for angle in angles:
        histogram, score = determine_score(thresh, angle)
        scores.append(score)

    best_angle = angles[scores.index(max(scores))]

    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, best_angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, \
              borderMode=cv2.BORDER_REPLICATE)

    return best_angle, rotated

In [7]:
##deskew 
##input binary image
## output deskewed image
img = cv2.imread(fileName, cv2.IMREAD_COLOR) 
image=np.copy(img)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray, 50, 200)
lines = cv2.HoughLinesP(edges, 1, np.pi/180, 100, minLineLength=10, maxLineGap=250)
for line in lines:

    x1, y1, x2, y2 = line[0]

    cv2.line(img, (x1, y1), (x2, y2), (255, 0, 0), 3)

display_image(img,"Lined image")

angle,rotated=correct_skew(image)
if abs(angle)>5:
    rotated=rotated
else:
    rotated=image
    
display_image(rotated,"rotated image")

print(angle)

0


In [ ]:
'''
image = cv2.imread(fileName)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray = 255 - gray
thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

coords = np.column_stack(np.where(thresh > 0))
angle = cv2.minAreaRect(coords)[-1]

if angle < -45:
    angle = -(90 + angle)
else:
    angle = -angle
print(angle)

if abs(angle)>10:
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    display_image(rotated,"Deskewed")
else:
    rotated=image

img = rotated
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray, 50, 200)
lines = cv2.HoughLinesP(edges, 1, np.pi/180, 100, minLineLength=10, maxLineGap=250)
for line in lines:

    x1, y1, x2, y2 = line[0]

    cv2.line(img, (x1, y1), (x2, y2), (255, 0, 0), 3)

display_image(img,"Lined image")
    
image = rotated
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray = 255 - gray
thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

coords = np.column_stack(np.where(thresh > 0))
angle = cv2.minAreaRect(coords)[-1]

if angle < -45:
    angle = -(90 + angle)
else:
    angle = -angle
print(angle)
'''

In [8]:
img = cv2.imread(fileName, 1)
img = cv2.cvtColor(rotated, cv2.COLOR_BGR2GRAY) #convert to gray scale

# applying Otsu thresholding 
# as an extra flag in binary  
# thresholding      
ret, img = cv2.threshold(img, 120, 255, cv2.THRESH_BINARY +  cv2.THRESH_OTSU) #otsu thresholding
ret,img = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY) #binary image gives 255 0 image
img = cv2.normalize(img, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)  #to get 0,1 image
binary_image=np.copy(img)
display_image(binary_image,"Binary image")


In [9]:
invert_image=np.ones(binary_image.shape)-binary_image
kernel=np.ones((3,3))
erosion_image = cv2.erode(invert_image, kernel,iterations=1) 

display_image(erosion_image,"Post inverting and eroding image ")


In [10]:
cv2.destroyAllWindows()

#input erosion image 
#output segmented character images and centroids, bounding boxes.

image_segement=np.copy(erosion_image)


image_segement=image_segement*255 #scales up the image values as we need a uint8 image for segmentation
image_segement = image_segement.astype(np.uint8) #convert the image to a uint8 from a float 64



contours, hier = cv2.findContours(np.copy(image_segement), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
contours = sorted(contours, key=lambda ctr: cv2.boundingRect(ctr)[0])
display_image(image_segement,"Contours")

d=0
segmented_chars=[]
for ctr in contours:
    # Get bounding box
    x, y, w, h = cv2.boundingRect(ctr)
    #getting centroid
    M = cv2.moments(ctr)
    cX = int(M["m10"] / M["m00"])
    cY = int(M["m01"] / M["m00"])
    # Getting ROI
    roi = image_segement[y:y+h, x:x+w]
    x,y,w,h = cv2.boundingRect(ctr)
    print(x,y,w,h,cX, cY)
#     
    blah=cv2.rectangle(np.copy(image_segement),(x,y),(x+w,y+h),(255,255,0),10)
    cv2.circle(blah, (cX, cY), 7, (255, 0, 0), -1)

    display_image(blah,'character: %d'%d)
    
    
    segmented_chars.append(Segments(roi,[x,y,w,h],[cX,cY]))
    
    cv2.imwrite('character_%d.png'%d, roi)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    d+=1
    
    

462 276 75 109 502 325
601 321 106 4 653 322
601 290 106 5 653 291
767 276 98 76 815 325
880 234 138 116 952 290
1075 255 106 106 1127 307
1238 234 118 116 1304 292


In [83]:
print(segmented_chars[0].image_val.shape)

(109, 75)


In [71]:
display_image(segmented_chars[0].image_val, "xvedgdgv")

In [ ]:
create_identifier(segmented_chars[0].image_val)

In [20]:
img=np.copy(segmented_chars[0].image_val)
img.shape
unique, counts = np.unique(img, return_counts=True)
dict(zip(unique, counts))

{0: 6463, 255: 1712}

In [54]:
img=np.ones(segmented_chars[0].image_val.shape)*255-np.copy(segmented_chars[0].image_val)

img = img/255
unique, counts = np.unique(img, return_counts=True)
print(dict(zip(unique, counts)))
k = 8;
#     IN2, R1-R7, D2-D8, 6 Hu Moments
identifier = np.zeros((1,2*k+6));
img_shape = img.shape
img_inv = np.ones(img_shape) - img
pad_size = 3
print((sum(img_inv[1,:])))

#     Top
if(sum(img_inv[0,:])==0):
    add = np.ones((pad_size,img_shape[1]))
    img = np.concatenate((add,img))
if(sum(img_inv[-1,:])==0) :#Bottom
    add = np.ones((pad_size,img_shape[1]))
    img = np.concatenate((img, add))

if(sum(img_inv[:,1])==0) :#left
    add = np.ones((img_shape[0], pad_size))
    img = np.hstack(( add, img))

if(sum(img_inv[:,1])==0) :#right
    add = np.ones((img_shape[0], pad_size))
    img = np.hstack(( img, add))
img_inv = np.ones(img_shape) - img


#Recalculte Centroid

img_inv = np.ones(img_shape) - img

img_shape = img.shape

N = np.sum(img==0)
x,y = np.where(img == 0)
x=x.reshape(x.shape[0],1)
y=y.reshape(y.shape[0],1)
y.shape


{0.0: 1712, 1.0: 6463}
8.0


(1712, 1)

In [55]:




#Find centroid 
#gib code kinshuk
M = cv2.moments(img_inv)
cent_x = int(M["m10"] / M["m00"])
cent_y = int(M["m01"] / M["m00"])
print(cent_x, cent_y)
blah = np.copy(img_inv)
cv2.circle(blah, (cent_x, cent_y), 7, (255, 0, 0), -1)

display_image(blah,'Inverse image centroids')

#     Find central moment of inertia
I = sum(np.squeeze((x - cent_x)**2 + (y - cent_y)**2));
IN2 = I / N**2;
identifier[0,0] = IN2
print(IN2)

x = img_shape[0]
y = img_shape[1]

dr = max(max(x - cent_x, cent_x), max(y - cent_y, cent_y))/(k+1)
c,r = np.meshgrid(np.arange(x),np.arange(y))
c = c + 1
r = r + 1
circvec = []

coding = {}

for i in range(1,k+1):
    rad = dr*i

    C = (((c - cent_x)**2 + (r - cent_y)**2)**0.5<=rad)^(((c - cent_x)**2 + (r - cent_y)**2)**0.5<=(rad-1))

#         print(np.count_nonzero((C+0) == 1))
    indx,indy = np.nonzero(C)
    indx = indx + 1
    indy = indy + 1
    cidx = (indx-1)*C.shape[0] + indy
    print("cidx", cidx)
    vals = []
    cx = c[indx-1,indy-1] - cent_x
    cy = r[indx-1,indy-1] - cent_y
    for m in range(len(cidx)):
        vals.append([cidx[m], cy[m], cx[m],0 ])
    
    vals = np.array(vals)
    
    
    
    TH,_ = (cart2pol(vals[:,1], vals[:,2]))
#         print(TH)
    vals[:,3] = TH

#         print(vals)

    sortedvals =  vals[np.argsort(vals[:, 3])]
    print(sortedvals)
    
    print("  ",C)
#     C1 = np.copy(C)
    
#     C1=
#     C1= C1.astype(np.uint8)
#     display_image(C1,"output")
    print(np.unravel_index(sortedvals[:,0], img.shape, 'F'))
    circvec = img[np.unravel_index(sortedvals[:,0], img.shape, 'F')]
    kernel = np.ones((2,1))

    print(circvec)

    circvec = cv2.morphologyEx(circvec, cv2.MORPH_OPEN, kernel)

    coding[i] = circvec

    if(len(circvec)!=0):
        circvec = np.squeeze(np.array(circvec))
        dummy = np.array([1,1])
        circvec = np.hstack((dummy, circvec))
        count = repr(circvec).count("0., 0.")

        identifier[0,i] = count
        circvec=circvec.reshape(len(circvec),1)

    
    circ=len(circvec)
    if (circvec[0]==1 and circvec[-1]==1):
        for z in range(len(circvec)-1,-1,-1):
            if circvec[z]==0:
                idx=z
                break
        
        print("idx",idx,circvec[idx+1:].shape,circvec[:idx+1].shape)
        circvec=np.vstack(((circvec[idx+1:]),(circvec[:idx+1])))
        
    B=np.hstack((np.zeros((1,1)),np.transpose(circvec)))
    B=np.hstack((B,np.zeros((1,1))))
    

    

li = cv2.HuMoments(cv2.moments(img_inv)).flatten()
print(li)       
print(identifier)

39 50
0.9015201245741986
cidx [3186 3187 3188 3189 3190 3191 3192 3260 3268 3334 3344 3408 3420 3482
 3496 3557 3571 3632 3646 3707 3721 3782 3796 3857 3871 3932 3946 4008
 4020 4084 4094 4160 4168 4236 4237 4238 4239 4240 4241 4242]
[[3186   -7   -3   -2]
 [3187   -7   -2   -2]
 [3188   -7   -1   -2]
 [3260   -6   -4   -2]
 [3334   -5   -5   -2]
 [3408   -4   -6   -2]
 [3932    3   -7   -1]
 [3857    2   -7   -1]
 [3782    1   -7   -1]
 [3632   -1   -7   -1]
 [3557   -2   -7   -1]
 [3482   -3   -7   -1]
 [3707    0   -7   -1]
 [4242    7    3    0]
 [4240    7    1    0]
 [4239    7    0    0]
 [4238    7   -1    0]
 [4237    7   -2    0]
 [4236    7   -3    0]
 [4241    7    2    0]
 [4168    6    4    0]
 [4160    6   -4    0]
 [4094    5    5    0]
 [4084    5   -5    0]
 [4020    4    6    0]
 [4008    4   -6    0]
 [3946    3    7    1]
 [3871    2    7    1]
 [3796    1    7    1]
 [3721    0    7    1]
 [3646   -1    7    1]
 [3571   -2    7    1]
 [3496   -3    7    1]
 [3190 